In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
import gymnasium as gym
import numpy as np
from sapien.core import Pose
import matplotlib.pyplot as plt
import os
import warnings
import shutil
import random
from plot_utils import *
from itertools import product, combinations, permutations

from utils import *
from agent_env_utils import *
from config import color_maps, background_quadrants
from scene_helpers import get_position_range, create_config, create_directories, finalize_quadrant


In [6]:
def get_ref_object_position(moving_obj_pos, direction, distance):
    diagonal_move = distance / math.sqrt(2)
    direction_vectors = {
        "left": np.array([0, distance]),
        "right": np.array([0, -distance]),
        "front": np.array([-distance, 0]), 
        "behind": np.array([distance, 0]),
        "left_front": np.array([-diagonal_move, diagonal_move]),
        "left_behind": np.array([diagonal_move, diagonal_move]),
        "right_front": np.array([-diagonal_move, -diagonal_move]),
        "right_behind": np.array([diagonal_move, -diagonal_move])
    }
    return moving_obj_pos[:2] + direction_vectors[direction]

def generate_non_overlapping_positions(moving_obj_pos, ref_obj_pos, num_positions, env, direction, distance):
    avoid_pos = [moving_obj_pos[:2], ref_obj_pos[:2]]
    
    # Generate path to avoid
    step = 0.01
    path = np.arange(0, 1 + step, step)
    for t in path:
        position = moving_obj_pos[:2] * (1 - t) + ref_obj_pos[:2] * t
        avoid_pos.append(position.tolist())
    
    positions = []
    for _ in range(num_positions):
        pos = generate_non_overlapping_position(avoid_pos, min_dist=0.1)
        positions.append(pos)
        avoid_pos.append(pos)
    
    return positions

def generate_scenes(env, final_dir, data_dir, early_stop=None):
    outputs = []
    scene_index = 0
    color_pairs = list(itertools.permutations(color_keys, 7))
    random.shuffle(color_pairs)
    color_pairs = color_pairs[:7]
    
    for background in range(4):
        for quadrant in background_quadrants[background]:
            for objects in object_pairs:
                for colors_pair in color_pairs:
                    for size in potential_sizes:
                        for direction in moving_directions:
                            scene_index += 1
                            if early_stop is not None and scene_index > early_stop:
                                return outputs
                            
                            # Generate main objects (moving and reference)
                            configs = []
                                
                            main_objects_meta = []
                            for obj_index, obj_type in enumerate(objects):
                                color = colors_pair[obj_index]
                                random.seed(scene_index*1000+obj_index)
                                position_range = get_position_range(background_quadrants, background, scale=0.6)
                                if obj_type == 'custom':
                                    obj_config = create_config(obj_type, color_maps[color], size*1.1/2, position_range, name="002_master_chef_can")
                                else:
                                    obj_config = create_config(obj_type, color_maps[color], size, position_range, name=obj_type)
                                configs.append(obj_config)
                                main_objects_meta.append({
                                    "type": obj_type,
                                    "color": color,
                                    "size": size if obj_type != 'custom' else size*1.1/2,
                                    "role": "moving" if obj_index == 0 else "reference"
                                })
                                
                            # Generate random background objects
                            num_bg_objects = random.randint(2, 4)
                            #bg_configs = []
                            background_objects_meta = []
                            for bg_idx in range(num_bg_objects):
                                bg_obj_type = random.choice(objects)
                                bg_color = colors_pair[len(objects) + bg_idx]
                                bg_size = random.uniform(0.02, 0.04)
                                position_range = get_position_range(background_quadrants, background, scale=0.6)
                                if bg_obj_type == 'custom':
                                    bg_config = create_config(bg_obj_type, color_maps[bg_color], bg_size*1.1/2, position_range, name="002_master_chef_can")
                                else:
                                    bg_config = create_config(bg_obj_type, color_maps[bg_color], bg_size, position_range, name=bg_obj_type)
                                configs.append(bg_config)
                                background_objects_meta.append({
                                    "type": bg_obj_type,
                                    "color": bg_color,
                                    "size": bg_size if bg_obj_type != 'custom' else bg_size*1.1/2,
                                })

                            configs_collate = collate_infos(configs)
                            env.register_configures(configs_collate)
                            _ = env.reset(options={"reconfigure": True})
                            print(len(configs))

                            initilization_poses, mov_pos, ref_pos = initialize_obj_nooverlap_path(direction, configs, direction_length, env)
                            initilization_poses[0] = mov_pos
                            initilization_poses[1] = ref_pos

                            env.set_poses(initilization_poses)

                            

                            

                            # configs_collate = collate_infos(configs)
                            # env.register_configures(configs_collate)
                            # _ = env.reset(options={"reconfigure": True})

                            # initialization_poses = env.get_important_obj_poses(mode="all")
                            # env.set_poses(initialization_poses)

                            direction_configs = [(0, 0, direction)]
                            distances = [direction_length]
                            frames = env.move_in_directions(direction_configs, distances, steps=None, camera_view=["top0"])
                            
                            video_name = f"scene_{scene_index}"
                            video_path = os.path.join(final_dir, f"videos/{video_name}.mp4")
                            os.makedirs(os.path.dirname(video_path), exist_ok=True)
                            generate_video(video_path, frames['top0'], fps=10, color_change=True)

                            image_dir = os.path.join(final_dir, f"images/scene_{scene_index}")
                            save_images_pararell(frames['top0'], image_dir)

                            outputs.append({
                                "scene_index": scene_index,
                                "source": os.path.relpath(video_path, start=data_dir),
                                "moving_obj": main_objects_meta[0],
                                "ref_obj": main_objects_meta[1],
                                "background_objects": background_objects_meta,
                                "direction": direction,
                                "distance": direction_length,
                                "background": background,
                                "quadrant": quadrant,
                            })
    
    return outputs

data_dir = "../benchmark/motion/direction"
final_dir = create_directories(data_dir, empty=True)

# Define parameters
objects = ["cube", "sphere", "custom"]
object_pairs = list(itertools.combinations(objects, 2))
moving_directions = ["front", "behind", "left", "right", "left_front", "left_behind", "right_front", "right_behind"]
color_keys = list(color_maps.keys())

potential_sizes = [0.03, 0.035, 0.04]
direction_length = 0.2

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    # Set early_stop to None if you want to generate all scenes
    early_stop = 10
    outputs = generate_scenes(env, final_dir, data_dir, early_stop)

    write_out(f"{data_dir}/index.jsonl", outputs)

del env

[2024-09-04 18:56:07.758] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.


4


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


5


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


...

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


5


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


4


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [6]:
[1,2] + [3]

[1, 2, 3]